In [2]:
import sys
import os
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
if project_root not in sys.path:
     sys.path.insert(0, project_root)

import json
from dotenv import load_dotenv
load_dotenv()
from supabase import create_client

from src.llm.tools.ChunkRetriever import RetrievalService
from src.llm.OpenAIClient import OpenAIClient
from src.storage.SupabaseService import SupabaseService

# ========== Constants and Environment Variables =========
YOUR_APP_DOMAIN = "www.stackifier.com"
TEST_EMAIL = os.environ.get("TEST_EMAIL")
TEST_PASSWORD = os.environ.get("TEST_PASSWORD")
if not TEST_EMAIL or not TEST_PASSWORD:
    raise ValueError("TEST_EMAIL and TEST_PASSWORD must be set in your .env file.")

supabase_url = os.environ.get("SUPABASE_URL")
supabase_key = os.environ.get("SUPABASE_ANON_KEY")
if not supabase_url or not supabase_key:
    raise ValueError("SUPABASE_URL and SUPABASE_ANON_KEY must be set in your .env file.")

auth_client_local = create_client(supabase_url, supabase_key)

print(f"Attempting to sign in with email: {TEST_EMAIL}")
auth_response = auth_client_local.auth.sign_in_with_password(
    {"email": TEST_EMAIL, "password": TEST_PASSWORD}
)
if not auth_response or not auth_response.user:
    error_detail = auth_response.error.message if hasattr(auth_response, 'error') and auth_response.error else "Unknown authentication error"
    raise ConnectionError(f"Supabase authentication failed: {error_detail}. Check credentials and Supabase Auth settings.")
authenticated_user_id_str_local = str(auth_response.user.id)
print(f"Authentication successful. User ID: {authenticated_user_id_str_local}")

user_given_id = authenticated_user_id_str_local

Attempting to sign in with email: user@example.com
Authentication successful. User ID: 372ec112-6fd7-46a7-bab4-abf7623fb05b


In [3]:
from src.llm.GeminiClient import GeminiClient
from src.llm.tools.FunctionCaller import call_function
from src.llm.tools.PythonCalculatorTool import PythonCalculationTool
from google.genai import types

# Define the function declaration for the model
python_calculator_tool_local = PythonCalculationTool()
python_calculator_decleration = python_calculator_tool_local.get_tool_declaration_data()

# Define the function to execute the Python code
execute_python_calculations = python_calculator_tool_local.execute_python_calculations
all_tools_functions = [
    execute_python_calculations,
]

# Define the function declaration for the model
all_tools_decleration = [
    types.Tool(function_declarations=[python_calculator_decleration]),
    types.Tool(function_declarations=[RetrievalService.get_tool_declaration()])
]

config = {
    "tools": all_tools_decleration,
    "automatic_function_calling": {"disable": True},
    "tool_config": {"function_calling_config": {"mode": "auto"}},
    "thinking_config": {"include_thoughts": True}
}

# prompt = "Show me annual report of tesla in 2021?" # "sum first 50 prime numbers, then divide by 0.04215215"
prompt = "Think first and use the retrieval tool. Whats the Gross Carrying Amount for Total intangible assets for tesla in 2021? then divide the value by 0.04215215"
prompt = "Whats 200525.52 * 20.5661614, think carefully, give me the answer in 4 decimal points. use 100 tokens. "
CONV_HISTORY = []
CONV_HISTORY.append(types.Content(role="user", parts=[types.Part(text=prompt)]))
print(CONV_HISTORY)
client = GeminiClient(config=config)#, model="gemini-2.0-flash")

[Content(parts=[Part(video_metadata=None, thought=None, code_execution_result=None, executable_code=None, file_data=None, function_call=None, function_response=None, inline_data=None, text='Whats 200525.52 * 20.5661614, think carefully, give me the answer in 4 decimal points. use 100 tokens. ')], role='user')]
Initializing Gemini client with API key: AIz...yQ


In [ ]:
import json
# ONLY WORKS WITH NON THIKIN MODEL
# Stream the entire response, capture the first chunk for debugging, and collect all text parts
texts = []
first_resp = None
client = GeminiClient(config=config)#, model="gemini-2.0-flash")
for chunk in client.generate_content_stream(contents=CONV_HISTORY):
    if first_resp is None:
        first_resp = chunk
    for cand in chunk.candidates:
        for part in cand.content.parts:
            if part.text:
                texts.append(part.text)

full_response = "".join(texts)
print("Full streamed response:\n", full_response)

# 4) inspect the very first chunk
if first_resp:
    resp_dict = first_resp.model_dump()
    if not resp_dict["candidates"][0]["content"]["parts"][0]["function_call"]:
        resp_dict["candidates"][0]["content"]["parts"][0]["text"] = full_response

    print("\n=== single response JSON ===")
    print(json.dumps(resp_dict, indent=4, default=str))

    content = resp_dict["candidates"][0]["content"]
    print("\n=== content only ===")
    print(json.dumps(content, indent=4, default=str))

Initializing Gemini client with API key: AIz...yQ
first_rep ][ candidates=[Candidate(content=Content(parts=[Part(video_metadata=None, thought=True, code_execution_result=None, executable_code=None, file_data=None, function_call=None, function_response=None, inline_data=None, text="**Calculating the Product**\n\nI've determined that `execute_python_calculations` is the ideal tool for the user's request. My plan is to execute the multiplication operation and subsequently format the outcome to four decimal places. The resulting value will be stored in `calculation_result`.\n\n\n")], role='model'), citation_metadata=None, finish_message=None, token_count=None, avg_logprobs=None, finish_reason=None, grounding_metadata=None, index=0, logprobs_result=None, safety_ratings=None)] model_version='models/gemini-2.5-flash-preview-05-20' prompt_feedback=None usage_metadata=GenerateContentResponseUsageMetadata(cached_content_token_count=None, candidates_token_count=None, prompt_token_count=650, total

In [ ]:
thoughts = ""
answer = ""
for chunk in client.generate_content_stream(
    contents=prompt,
):
  for part in chunk.candidates[0].content.parts:
    if not part.text:
      print("function:", part.text)
      continue
    elif part.thought:
      if not thoughts:
        print("Thoughts summary:")
      print(part.text)
      thoughts += part.text
    else:
      if not answer:
        print("Thoughts summary:")
      print(part.text)
      answer += part.text
    if part.function_call:
      print(part.text) # HOW TO GET SECOND PART? The function call?

Thoughts summary:
**Performing the Calculation**

I've determined that the `execute_python_calculations` tool is the best fit for this task. My current focus is on performing the multiplication and then formatting the output to precisely four decimal places, as requested. I'm aiming for accuracy and efficiency.



function: None
